In [1]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_ChanRed
import torch
import os

In [4]:
model = EEGNet(nb_classes = 2, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  50946


In [3]:
model_SEED = EEGNet_ChanRed(nb_classes = 3, Chans = 62, InnerChans=14, Samples = 200, dropoutRate = 0.1, kernLength = 25, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model_SEED.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  53045


In [4]:
x = torch.randn(128, 1, 62, 200)
out = model_SEED(x)
print(out.shape)

torch.Size([128, 3])


/users/eleves-a/2021/julien.gadonneix/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1040.)
  return F.conv2d(input, weight, bias, self.stride,


In [5]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

16
1
cuda
True
_CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24218MB, multi_processor_count=64)


In [6]:
import subprocess
import plistlib

def get_gpu_info():
    """Get information about the GPU on macOS."""
    # Run system_profiler command and capture the output
    result = subprocess.run(['system_profiler', '-xml', 'SPDisplaysDataType'], capture_output=True)
    if result.returncode == 0:
        # Parse the XML output
        display_info = plistlib.loads(result.stdout)
        # Extract GPU information
        gpu_count = len(display_info[0]['_items'])
        return gpu_count
    else:
        print("Error:", result.stderr)
        return None

gpu_count = get_gpu_info()
if gpu_count is not None:
    print("Number of GPUs:", gpu_count)


FileNotFoundError: [Errno 2] No such file or directory: 'system_profiler'

In [ ]:
import time
import torch

start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.3293938636779785


In [ ]:
t = torch.randn(8,8)
print(t)
torch.reshape(t, (-1, 2, 8))

tensor([[-1.2217e+00,  1.0289e+00,  6.0849e-01,  1.0410e+00,  8.6184e-01,
         -2.1006e+00, -7.6635e-01,  1.3263e+00],
        [-8.3535e-01, -4.7972e-01,  1.1489e+00, -1.6138e+00,  5.5229e-01,
         -1.0601e+00, -2.6549e+00, -8.9108e-01],
        [ 5.6502e-01,  4.7087e-01, -5.1239e-01, -5.0209e-01,  7.7523e-04,
          1.0167e+00, -6.3848e-01, -1.9583e-01],
        [-2.6021e-01, -1.0502e+00, -5.2712e-01,  5.9923e-01,  1.6047e+00,
         -1.4371e+00, -4.2346e-01,  6.7980e-01],
        [ 2.3133e+00, -8.8326e-01,  4.7351e-01,  3.2216e-03,  1.8865e-02,
         -8.5215e-01,  9.9883e-01,  7.7687e-01],
        [ 9.5250e-01,  1.8493e+00, -2.2837e-02,  8.3958e-01, -1.4960e+00,
          1.3575e+00, -5.3816e-01, -1.8288e+00],
        [ 3.0960e-02,  2.7033e+00, -1.7205e+00,  1.9184e+00, -4.0058e-01,
         -9.0875e-01,  1.2706e-01, -2.2948e+00],
        [ 5.0333e-01,  1.0464e+00, -1.9660e+00, -1.2091e+00, -1.7362e+00,
          2.6974e-01, -6.7245e-02,  5.8963e-02]])


tensor([[[-1.2217e+00,  1.0289e+00,  6.0849e-01,  1.0410e+00,  8.6184e-01,
          -2.1006e+00, -7.6635e-01,  1.3263e+00],
         [-8.3535e-01, -4.7972e-01,  1.1489e+00, -1.6138e+00,  5.5229e-01,
          -1.0601e+00, -2.6549e+00, -8.9108e-01]],

        [[ 5.6502e-01,  4.7087e-01, -5.1239e-01, -5.0209e-01,  7.7523e-04,
           1.0167e+00, -6.3848e-01, -1.9583e-01],
         [-2.6021e-01, -1.0502e+00, -5.2712e-01,  5.9923e-01,  1.6047e+00,
          -1.4371e+00, -4.2346e-01,  6.7980e-01]],

        [[ 2.3133e+00, -8.8326e-01,  4.7351e-01,  3.2216e-03,  1.8865e-02,
          -8.5215e-01,  9.9883e-01,  7.7687e-01],
         [ 9.5250e-01,  1.8493e+00, -2.2837e-02,  8.3958e-01, -1.4960e+00,
           1.3575e+00, -5.3816e-01, -1.8288e+00]],

        [[ 3.0960e-02,  2.7033e+00, -1.7205e+00,  1.9184e+00, -4.0058e-01,
          -9.0875e-01,  1.2706e-01, -2.2948e+00],
         [ 5.0333e-01,  1.0464e+00, -1.9660e+00, -1.2091e+00, -1.7362e+00,
           2.6974e-01, -6.7245e-02,  5.8963e